In [32]:
import pandas as pd
import numpy as np
import os
import datetime
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import bar_chart_race as bcr

In [33]:
path = "../COVID19/World/WorldSummary/"
continents  =  ['Africa', 'Asia', 'Europe', 'North America', 'Oceania', 'South America']
cases = ['Total_Cases', 'New_Cases', 'Total_Deaths', 'New_Deaths', 'Total_Recovered', 'Active_Cases']

In [34]:
given_date = pd.read_csv('../COVID19/World/WorldSummary/Europe/Europe_summary.csv').iloc[-1]['Date']


if '_' in given_date:
    given_date = given_date[:-7] + ' '+ given_date[-7:-5] + ', '+ given_date[-4:] 
else:
    given_date = given_date[:-2] + ' '+ given_date[-2:] + ', '+ str(2020)
        
    
given_date

'October 14, 2020'

In [35]:
fig = go.Figure()
for continent in continents:
    continent_mod = continent.replace(" ", "")
    full_path = path + continent_mod + '/'+ continent_mod +'_summary.csv'
    summary = pd.read_csv(full_path)
    fig.add_trace(go.Scatter(
        x=summary['Date'].tolist(),
        y= summary['Total_Cases'].tolist(),
#         y = total_cases,
        name=continent,
        mode='lines',
        line=dict(width=3)
        ))   
fig.update_layout(
    template='plotly_dark',        
    title={
#         'text': '<b>'+'WORLD COVID19 CONFIRMED CASES IN AFRICA: ' + date_string +'</b>',
        'text': '<b>'+'WORLD COVID19 CONFIRMED CASES BY CONTINENT: ' + given_date + '</b>',
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
    xaxis=dict(title='<b>Date</b>',titlefont_size=18, titlefont=dict(color='#4D4B4B'),nticks=30),
    yaxis=dict(title='<b>Confirmed Cases</b>',titlefont_size=18,  titlefont=dict(color='#BDBDBD')),
    legend=dict(x=0.07, y=0.98,traceorder="normal",bgcolor='rgba(255, 255, 255, 0)',bordercolor='rgba(255, 255, 255, 0)',),   
    xaxis_tickangle=90
)
fig.show()

In [36]:
fig = make_subplots(rows=3, cols=2,  specs=[[{'type':'domain'}, {'type':'domain'}],[{'type':'domain'}, {'type':'domain'}],
                                            [{'type':'domain'}, {'type':'domain'}]], print_grid=False,vertical_spacing=0.005,
                                            horizontal_spacing=0.085)

case_val =  []
# slice_colors =  ['crimson', 'green', 'saddlebrown', 'yellow', 'cyan','blue' ]
slice_colors =  ['gold', 'sapphire', 'green', 'saddlebrown', 'cyan','crimson' ]
position = 0
x=[0.168, 0.82, 0.168, 0.82, 0.148, 0.82]
y=[0.84,0.84, 0.5,0.5,0.155, 0.155]
for case in cases:
    row = ((position//2)+1)
    col = ((position%2)+1)
    for continent in continents:
        continent_mod = continent.replace(" ", "")
        full_path = path + continent_mod + '\\'+ continent_mod +'_summary.csv'
        summary = pd.read_csv(full_path)
        case_val.append(summary.iloc[-1][case])
        case_mod = case.replace('_', '\n')
        
    fig.add_trace(go.Pie(labels=continents, values=case_val,name=case, ),row, col) 
    fig.update_traces(hole=.35, hoverinfo="label+percent+name+value", textinfo='percent+value',textfont_size=15,marker_colors=slice_colors,)         
    case_val = []
    fig['layout']['annotations'] += tuple([dict(x=x[position],y=y[position],text='<b>'+case_mod+'</b>', showarrow=False, font_size=12, font=dict(color='black'))])
    fig.update_layout(
    width=990,height=1450,autosize=False,
#     title={'text': '<b>' + 'AFRICA - COVID19 NEW CASES BY REGION: ' + date_string +'</b>',
    title={'text': '<b>' + 'WORLD - COVID19 SUMMARY BY CONTINENT: ' + given_date + '</b>',       
    'y':0.95,
    'x':0.46,
    'xanchor': 'center',
    'yanchor': 'top',
    'font_color':'black',
    'font_size':18},
     legend=dict(x=1, y=1,traceorder="normal",  font=dict(size=13))
    )
    position=position+1  
fig.show()    
    
    


## Countries With Zero Case in the Past One Month

In [37]:
file_path = "../COVID19/World/NewCases"
files = os.listdir(file_path)
latest_new_case = [file for file in files if file[-3:] == 'csv' and file[:10] == 'DailyCases']
latest_new_case

['DailyCases14102020.csv']

In [41]:
last_new_case_record = pd.read_csv('../COVID19/World/LastNewCaseReport/LastNewCaseReport.csv', dtype=object, encoding='utf-8')
last_new_case_record.tail()

,Country_Other,Total_Cases,Total_Recovered,Total_Deaths,Active_Cases,New_Cases,Last_New_Case_Report_Date
211,St. Barth,67,55,0,12,+2,14-Oct-2020
212,Western Sahara,10,8,1,1,0,22-Jun-2020
213,Anguilla,3,3,0,0,0,2-Apr-20
214,Lesotho,"1,833",961,42,830,+11,14-Oct-2020
215,Saint Pierre Miquelon,16,12,0,4,0,24-Sep-2020


In [39]:
last_new_case_record['last_case'] = last_new_case_record['Last_New_Case_Report_Date']
last_new_case_record['last_case'] = pd.to_datetime(last_new_case_record['last_case'])

date_string = latest_new_case[0]

date_string = date_string[10:18]

date_string = datetime.datetime.strptime(date_string, '%d%m%Y')

last_new_case_record['number_of_days'] =  date_string - last_new_case_record['last_case']

last_new_case_record['days'] = last_new_case_record['number_of_days'].apply(lambda x : str(x).split(' ')[0])

last_new_case_record['days'] = last_new_case_record['days'].astype(int)
last_new_case_record['days']
upto_a_month_free = last_new_case_record[last_new_case_record['days'] >= 30]
upto_a_month_free = upto_a_month_free.copy()
diamond_princes = upto_a_month_free.index[upto_a_month_free['Country_Other'] == 'Diamond Princess'][0]
ms_zaandam = upto_a_month_free.index[upto_a_month_free['Country_Other'] == 'MS Zaandam'][0]
upto_a_month_free.drop(['New_Cases','last_case','days'], axis=1, inplace=True)
upto_a_month_free.drop([diamond_princes, ms_zaandam],  inplace=True)
upto_a_month_free.reset_index(drop=True, inplace=True)
upto_a_month_free

,Country_Other,Total_Cases,Total_Recovered,Total_Deaths,Active_Cases,Last_New_Case_Report_Date,number_of_days
0,Tanzania,509,183,21,305,8-May-20,159 days
1,Macao,46,46,0,0,26-Jun-2020,110 days
2,Laos,23,22,0,1,11-Sep-2020,33 days
3,Fiji,32,30,2,0,09-Sep-2020,35 days
4,St. Vincent Grenadines,64,64,0,0,12-Sep-2020,32 days
5,Falkland Islands,13,13,0,0,25-Apr-20,172 days
6,Montserrat,13,12,1,0,11-Jul-2020,95 days
7,Western Sahara,10,8,1,1,22-Jun-2020,114 days
8,Anguilla,3,3,0,0,2-Apr-20,195 days


In [42]:
continent_summary = pd.read_csv('../COVID19/World/WorldSummary/SouthAmerica/SouthAmerica_summary.csv')
continent_summary
all_continents_new_cases = pd.DataFrame()
all_continents_new_cases['Date'] = continent_summary['Date']
for continent in continents:
        continent_mod = continent.replace(" ", "")
        full_path = path + continent_mod + '\\'+ continent_mod +'_summary.csv'
        summary = pd.read_csv(full_path)
        all_continents_new_cases[continent] = summary['Total_Cases']

all_continents_new_cases = all_continents_new_cases.reset_index(drop=True)
all_continents_new_cases=all_continents_new_cases.set_index('Date')
bcr.bar_chart_race(all_continents_new_cases,title='World COVID-19 Confirmed cases')